In [34]:
import json
from io import open

def preprocess_data(file_path):
    train_data = {}

    with open(file_path, 'r') as file:
        i = 0
        for line in file:
            # Parse the line as JSON
            train_data[i] = json.loads(line)
            i+=1

    inputs, targets = [], []

    for key, instance in train_data.items():
        if isinstance(instance, dict) and 'messages' in instance:
            messages = instance['messages']
            game_scores = instance['game_score']

        for idx, (message, score) in enumerate(zip(messages, game_scores)):
            start_index = max(0, idx-5)
            end_index = min(len(messages)-1, idx)
            context = " ".join(messages[start_index:end_index])
            inputs.append(f"Given the following context, generate an appropriate repsonse: {context}")
            targets.append(message)
    return inputs, targets

training_inputs, training_targets = preprocess_data("2020_acl_diplomacy/data/train.jsonl")
validation_inputs, validation_targets = preprocess_data("2020_acl_diplomacy/data/validation.jsonl")
test_inputs, test_targets = preprocess_data("2020_acl_diplomacy/data/test.jsonl")

print(len(training_inputs)) 
print(len(training_targets))
print(training_inputs[3])
print(training_targets[3])

13132
13132
Given the following context, generate an appropriate repsonse: Germany!

Just the person I want to speak with. I have a somewhat crazy idea that I’ve always wanted to try with I/G, but I’ve never actually convinced the other guy to try it. And, what’s worse, it might make you suspicious of me. 

So...do I suggest it?

I’m thinking that this is a low stakes game, not a tournament or anything, and an interesting and unusual move set might make it more fun? That’s my hope anyway.

What is your appetite like for unusual and crazy? You've whet my appetite, Italy. What's the suggestion? 👍
It seems like there are a lot of ways that could go wrong...I don't see why France would see you approaching/taking Munich--while I do nothing about it--and not immediately feel skittish


In [35]:
%pip install transformers
%pip install torch

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

class DiplomacyDataset(Dataset):
    def __init__(self, data, targets, tokenizer, max_len=512):
        self.data = data
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        tokenized_input = self.tokenizer.encode(self.data[index], max_length=self.max_len, truncation=True, padding='max_length', return_tensors='pt')
        tokenized_target = self.tokenizer.encode(self.targets[index], max_length=self.max_len, truncation=True, padding='max_length', return_tensors='pt')
        item = {
            'input_ids': tokenized_input.flatten(),
            # 'decoder_input_ids': tokenized_target.flatten()[:-1], # remove the last token of the target
            'labels': tokenized_target.flatten()[1:],
        }
        return item

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

train_dataset = DiplomacyDataset(training_inputs, training_targets, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

validation_dataset = DiplomacyDataset(validation_inputs, validation_targets, tokenizer)
validation_loader = DataLoader(validation_dataset, batch_size=4, shuffle=True)


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [36]:
%pip install accelerate -U
import accelerate


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-4)

for epoch in range(3):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")
        


# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     num_train_epochs=3,
#     learning_rate=3e-4,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=validation_dataset
# )



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Epoch: 0, Loss: 2.521228790283203
Epoch: 0, Loss: 2.0196661949157715
Epoch: 0, Loss: 2.0782434940338135
Epoch: 0, Loss: 1.8016599416732788
Epoch: 0, Loss: 1.8211919069290161
Epoch: 0, Loss: 1.907079815864563
Epoch: 0, Loss: 1.5171607732772827
Epoch: 0, Loss: 1.6586593389511108
Epoch: 0, Loss: 1.2573260068893433
Epoch: 0, Loss: 1.2407628297805786
Epoch: 0, Loss: 0.9981366395950317
Epoch: 0, Loss: 0.9253311157226562
Epoch: 0, Loss: 0.6820459365844727
Epoch: 0, Loss: 0.6176517009735107
Epoch: 0, Loss: 1.1860613822937012
Epoch: 0, Loss: 0.5721552968025208
Epoch: 0, Loss: 0.3700699210166931
Epoch: 0, Loss: 0.6948256492614746
Epoch: 0, Loss: 0.31539386510849
Epoch: 0, Loss: 0.26961666345596313
Epoch: 0, Loss: 0.38584816455841064
Epoch: 0, Loss: 0.24448588490486145
Epoch: 0, Loss: 0.942680835723877
Epoch: 0, Loss: 0

KeyboardInterrupt: 

In [ ]:
# trainer.train()

  0%|          | 10/3283 [00:18<1:05:46,  1.21s/it]

{'loss': 3.1059, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.0}


  1%|          | 20/3283 [00:29<59:55,  1.10s/it]  

{'loss': 2.4386, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.01}


  1%|          | 30/3283 [00:40<1:00:07,  1.11s/it]

...

100%|█████████▉| 3280/3283 [1:22:46<00:03,  1.04s/it]

{'loss': 0.1886, 'learning_rate': 2.1559468199784405e-08, 'epoch': 1.0}


                                                     
100%|██████████| 3283/3283 [1:23:41<00:00,  1.53s/it]

{'eval_loss': 0.18469354510307312, 'eval_runtime': 51.2079, 'eval_samples_per_second': 27.652, 'eval_steps_per_second': 6.913, 'epoch': 1.0}
{'train_runtime': 5021.0636, 'train_samples_per_second': 2.615, 'train_steps_per_second': 0.654, 'train_loss': 0.42550153326341883, 'epoch': 1.0}


TrainOutput(global_step=3283, training_loss=0.42550153326341883, metrics={'train_runtime': 5021.0636, 'train_samples_per_second': 2.615, 'train_steps_per_second': 0.654, 'train_loss': 0.42550153326341883, 'epoch': 1.0})

In [ ]:
model.eval()
input_ids = validation_dataset[10]['input_ids'].unsqueeze(0).to(device)
outputs = model.generate(input_ids)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
